In [1]:
from indox import IndoxRetrievalAugmentation
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding
from indox.data_loader_splitter.SimpleLoadAndSplit import SimpleLoadAndSplit

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [ ]:
indox = IndoxRetrievalAugmentation()
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully


.gitattributes:   0%|          | 0.00/791 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

data_config.json:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
file_path = "sample.txt"


In [5]:
simpleLoadAndSplit = SimpleLoadAndSplit(file_path="sample.txt",remove_sword=False,max_chunk_size=200)
docs = simpleLoadAndSplit.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Starting processing
INFO: Created initial document elements
INFO: Completed chunking process
INFO: Successfully obtained all documents


In [6]:
docs

["The wife of a rich man fell sick, and as she felt that her end was drawing near, she called her only daughter to her bedside and said, dear child, be good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you.  Thereupon she closed her eyes and departed.  Every day the maiden went out to her mother's grave, and wept, and she remained pious and good.  When winter came the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife. The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart. Now began a bad time for the poor step-child.  Is the stupid goose to sit in the parlor with us, they said.  He who wants to eat bread",
 'must earn it.  Out with the kitchen-wench.  They took her pretty clothes away from her, put an old grey bedgown on her, and gave her wooden shoes.  Just look at the prou

In [7]:
from indox.vector_stores import MongoDB
db = MongoDB(collection_name="sample",embedding_function=embed)

In [9]:
db.add(docs=docs)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


In [10]:
query = "How cinderella reach her happy ending?"


In [11]:
retriever = indox.QuestionAnswer(vector_database=db,llm=mistral_qa,top_k=5)


In [12]:
answer = retriever.invoke(query=query)


INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [13]:
answer

"Cinderella reached her happy ending through the help of a magical little white bird. She sat beneath a tree and cried, prayed, and wished for what she desired. The bird granted her requests by throwing down what she had wished for. During a three-day festival at the king's palace, Cinderella used the bird's magic to obtain a golden dress and slippers, allowing her to attend the event incognito. The prince fell in love with her and didn'"

In [14]:
context = retriever.context
context

['must earn it.  Out with the kitchen-wench.  They took her pretty clothes away from her, put an old grey bedgown on her, and gave her wooden shoes.  Just look at the proud princess, how decked out she is, they cried, and laughed, and led her into the kitchen. There she had to do hard work from morning till night, get up before daybreak, carry water, light fires, cook and wash.  Besides this, the sisters did her every imaginable injury - they mocked her and emptied her peas and lentils into the ashes, so that she was forced to sit and pick them out again.  In the evening when she had worked till she was weary she had no bed to go to, but had to sleep by the hearth in the cinders.  And as on that account she always looked dusty and dirty, they called her cinderella. It happened that the father was once going to the fair, and he',
 "And now the bird threw down to her a dress which was more splendid and magnificent than any she had yet had, and the slippers were golden.  And when she went

In [15]:
db.__len__()

17

In [16]:
db.get()

{'ids': ['057d5a96-a42d-44c4-8fc5-fa8e8968dc3a',
  '61b9e98d-132e-4229-b01b-ed5146de905e',
  '8c90684d-dd41-485a-abd3-e9244a630fb5',
  '4fbcb91b-b794-4e2f-b1d4-ef91eec06f45',
  'da1c7ca6-d7e5-4e24-8afa-a111d1f0141a',
  'bf3e8d1f-8cb8-4196-a45c-e6f0a1561f54',
  '38243480-4579-4a91-b575-c3dc18fd1f56',
  '4a0562f3-7fe4-4331-a8ad-1aab574dc9d5',
  'e745545d-2e3a-422b-8f0f-98f2c8e67c82',
  'b1b801ab-fdf5-48ea-8627-90fb1dffa9ee',
  '9a09db5c-5a94-4305-81da-4d4c97c5b058',
  'f11f0943-36b4-4210-b220-6efe2da7e237',
  '0a8914f7-c44e-4abd-ad4f-2fad4622539f',
  'c09283e4-8d26-4aea-ae06-ff79a7cfb8cf',
  '20028569-98bf-4886-ba72-72f8636ed0f3',
  'a02f593a-6159-465e-b809-88336954ea2b',
  'de9f720f-d928-43b9-a267-0d0f40a604df'],
 'embeddings': None,
 'metadata': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'documents': ["The wife of a rich man fell sick, and as she felt that her end was drawing near, she called 

In [17]:
# db.delete_collection()